In [90]:
import pandas as pd

In [91]:
df1 = pd.read_csv('./data/input/annual_visitors.csv', sep=';')
df2 = pd.read_csv('./data/input/annual_visitors_air.csv', sep=';')
df3 = pd.read_csv('./data/input/age_group.csv')

In [92]:
df3_m = pd.melt(df3, id_vars=['age_group']).rename({'variable': 'year', 'age_group': 'variable'}, axis=1).assign(category='Age Groups')

In [93]:
df3_m['year'] = df3_m['year'].str[1:].astype(int)

In [94]:
df = df1.merge(df2[['Country', 'Continente']].drop_duplicates(), left_on='country', right_on='Country', how='left')

In [95]:
df['air_way'] = df['air_way'].div(100).mul(df['annual_visitors'])
df['sea_way'] = df['sea_way'].div(100).mul(df['annual_visitors'])
df['rail_way'] = df['rail_way'].div(100).mul(df['annual_visitors'])
df['land_way'] = df['land_way'].div(100).mul(df['annual_visitors'])

df['male_visitors'] = df['male_visitors'].div(100).mul(df['annual_visitors'])
df['female_visitors'] = df['female_visitors'].div(100).mul(df['annual_visitors'])

In [96]:
df_g = df.groupby(['year'], as_index=False)[[
    'air_way',
    'sea_way',
    'rail_way',
    'land_way',
    'male_visitors',
    'female_visitors',
]].sum()

In [97]:
df_c_g = df.groupby(['year', 'Continente'], as_index=False)[[
    'annual_visitors'
]].sum()

In [98]:
df_m = pd.melt(df_g, id_vars=['year'])

In [99]:
df_c_m = pd.melt(df_c_g, id_vars=['year', 'Continente']).drop('variable', axis=1).rename({'Continente': 'variable'}, axis=1).assign(category='Continent')

In [100]:
df_m['variable'] = df_m['variable'].str.replace('_', ' ').str.capitalize()

In [101]:
import numpy as np

df_m['category'] = np.where(
    df_m['variable'].str.contains(' way'),
    'Transport mean',
    'Sex'
)

In [102]:
df_f = pd.concat([df_m, df_c_m, df3_m])

In [103]:
df_f

,year,variable,value,category
0,2014,Air way,7090670.499,Transport mean
1,2015,Air way,7392856.960,Transport mean
2,2016,Air way,7111654.142,Transport mean
3,2017,Air way,7667794.071,Transport mean
4,2018,Air way,8064517.823,Transport mean
...,...,...,...,...
44,2020,25-34,1344.500,Age Groups
45,2020,35-44,1330.300,Age Groups
46,2020,45-54,1230.900,Age Groups
47,2020,55-64,959.400,Age Groups


In [73]:
df_f.to_csv('yearly_categories.csv')